In [1]:
import datetime
from os.path import join

from script.core.transformers import *
from script.models.cnn import CNN
from script.core.models import *
from script.core.dataset import Watcher
from script.core.utils import read_dataset, get_result, logging
from script.core.pipeline import Pipeline

# linear models
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# sklearn feachure extractors
from sklearn.feature_extraction.text import CountVectorizer as count
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.preprocessing import LabelEncoder

# Pure DilyaraModel
# from intent_classifier.intent_model.model_wrap import KerasMulticlassModel as DilyaraModel
# from script.models.cnn import CNN
# from script.models.dcnn import DCNN

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/mks/envs/intent_script/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
2018-04-03 12:41:14.375 DEBUG in 'matplotlib.backends'['__init__'] at line 90: backend module://ipykernel.pylab.backend_inline version unknown


# Read csv file and create Dataset

In [2]:
def init_dataset_tiny(file_path, language, dataset_name, date, seed=42):
    pure_data = read_dataset(file_path, True, True)  # It not default meanings!!!
    start_dataset = Watcher(pure_data, date, language, dataset_name,
                            seed=seed)  # classes_descriptions = {} we can do it

    ######################################################################################
#     dataset = start_dataset.split([0.1, 0.1])
#     data = dataset.data['test']
#     start_dataset = Watcher(data, date, language, dataset_name, seed)
    ######################################################################################

    return start_dataset

date = datetime.datetime.now()
dataset_name = 'vkusvill'
language = 'russian'
file_path = join('./data', language, dataset_name, 'data', 'vkusvill_all_categories.csv')

dataset = init_dataset_tiny(file_path, language, dataset_name, date)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Operations

In [3]:
spl_conf = {'op_type': 'transformer',
            'name': 'Speller',
            'request_names': ['base'],
            'new_names': ['base'],
            'path': './DeepPavlov/deeppavlov/configs/error_model/brillmoore_kartaslov_ru.json'}

tok_conf = {'op_type': 'transformer',
            'name': 'Tokenizer',
            'request_names': ['base'],
            'new_names': ['base']}

lem_conf = {'op_type': 'transformer',
            'name': 'Lemmatizer',
            'request_names': ['base'],
            'new_names': ['base']}

concat = TextConcat()

tfidf_conf_1 = {'op_type': 'vectorizer', 'name': 'tf-idf vectorizer',
                'request_names': ['train', 'valid', 'test'], 'new_names': ['train_vec', 'valid_vec', 'test_vec']}
tfidf_conf_2 = {'op_type': 'vectorizer', 'name': 'count_vectorizer',
                'request_names': ['train', 'valid', 'test'], 'new_names': ['train_vec', 'valid_vec', 'test_vec']}
tfidf_ = sktransformer(tfidf, tfidf_conf_1)
count_ = sktransformer(count, tfidf_conf_2)

# Pipeline with Neural Models

## Run

In [4]:
class GetCNN(BaseModel):
    def init_model(self, dataset):
        classes = dataset.get_classes()
        classes = ' '.join([str(x) for x in classes])
        self.config['classes'] = classes
        
        super().init_model(dataset)
        
        return self

def get_cnn_conf(path, emb_path, fit_names=None, predict_names=None, new_names=None):
    with open(path, 'r') as conf:
        config = json.load(conf)

    config['op_type'] = 'model'
    config['name'] = 'cnn'
    
    if fit_names is not None:
        config['fit_names'] = fit_names
    else:
        config['fit_names'] = ['train_vec']
    
    if predict_names is not None:
        config['predict_names'] = predict_names
    else:
        config['predict_names'] = ['test_vec']
        
    if new_names is not None:
        config['new_names'] = new_names
    else:
        config['new_names'] = ['predicted_test']
    
    config['fasttext_model'] = emb_path
    
    return config

path_0 = './configs/models/CNN/CNN_opt.json'
path_1 = './configs/models/CNN.json'
emb_path = './data/russian/embeddings/ft_0.8.3_nltk_yalen_sg_300.bin'

In [5]:
# config = get_cnn_conf(path_0, emb_path, fit_names=['train'], predict_names=['test'])
# dataset = init_dataset()
# Dilyara = GetCNN(DilyaraModel, config)
# neuro_pipe = [(Tokenizer, ), (Lemmatizer,), (TextConcat,), (Dilyara,), (GetResult,)]
# pipeline_0 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_0.run(dataset)

In [6]:
config = get_cnn_conf(path_1, emb_path)
WCNN = GetCNN(CNN, config)
neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (WCNN,)]
pipeline_1 = Pipeline(neuro_pipe, mode='train', output=None)
neurodata = pipeline_1.run(dataset)

  1%|          | 510/58780 [00:00<00:11, 5099.06it/s]

[ Starting tokenization ... ]


100%|██████████| 58780/58780 [00:09<00:00, 5900.83it/s]


[ Tokenization was done. ]


  0%|          | 243/49830 [00:00<00:20, 2421.62it/s]

...

[ Vectorization of valid part of dataset ... ]


  4%|▍         | 185/4475 [00:00<00:02, 1849.04it/s]

[ Vectorization of test part of dataset ... ]


100%|██████████| 4475/4475 [00:02<00:00, 2045.84it/s]


[ Vectorization was ended. ]
[ Initializing intent_model from scratch ]

____Training over 49830 samples____


train -->	updates: 1	loss: 0.3408034145832062	fmeasure: 0.02941175550222397	 
train -->	updates: 501	loss: 0.13379965722560883	fmeasure: 0.6285713911056519	 
epochs_done: 1
train -->	updates: 780	loss: 0.1236955001950264	fmeasure: 0.6428570747375488	 
train -->	updates: 1280	loss: 0.15532679855823517	fmeasure: 0.5656565427780151	 
epochs_done: 2
train -->	updates: 1559	loss: 0.11766139417886734	fmeasure: 0.7047618627548218	 
train -->	updates: 2059	loss: 0.11040987819433212	fmeasure: 0.5858585238456726	 
epochs_done: 3
train -->	updates: 2338	loss: 0.09717939794063568	fmeasure: 0.7079644799232483	 
train -->	updates: 2838	loss: 0.11789267510175705	fmeasure: 0.6285713911056519	 
epochs_done: 4
train -->	updates: 3117	loss: 0.11459743976593018	fmeasure: 0.7256636023521423	 
train -->	updates: 3617	loss: 0.08892318606376648	fmeasure: 0.770642101764679	 
epochs_done: 5


In [7]:
model = pipeline_1.get_last_model()

In [8]:
model = model[1]

In [9]:
pipe_ = [(Tokenizer, ), (FasttextVectorizer,)]
pipe = Pipeline(pipe_)

dataset = init_dataset_tiny(file_path, language, dataset_name, date)
data = pipe.run(dataset)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
  2%|▏         | 1128/58780 [00:00<00:10, 5631.33it/s]

[ Starting tokenization ... ]


100%|██████████| 58780/58780 [00:09<00:00, 6066.58it/s]


[ Tokenization was done. ]


  0%|          | 229/49830 [00:00<00:22, 2248.14it/s]

...

[ Vectorization of valid part of dataset ... ]


  4%|▍         | 187/4475 [00:00<00:02, 1862.79it/s]

[ Vectorization of test part of dataset ... ]


100%|██████████| 4475/4475 [00:02<00:00, 2017.52it/s]


[ Vectorization was ended. ]


In [10]:
data

In [11]:
pred_data = model.predict(data)

In [12]:
pred_data.data.keys()

dict_keys(['train', 'valid', 'test', 'train_vec', 'valid_vec', 'test_vec', 'predicted_test'])

In [13]:
pred_data.data['predicted_test'][0].shape

(64, 17)

In [14]:
pred_data.data['predicted_test'][0][0]

array([0.00670387, 0.01195906, 0.21842773, 0.11691485, 0.00080399,
       0.00406716, 0.5889138 , 0.00868505, 0.00401698, 0.00794192,
       0.01409593, 0.00502214, 0.0046623 , 0.00147805, 0.00124307,
       0.00431627, 0.0007478 ], dtype=float32)

In [15]:
pred_data.data['predicted_test'][0][0].max()

0.5889138

In [16]:
argmax = np.argmax(pred_data.data['predicted_test'][0], axis=1)

In [17]:
argmax

array([ 6,  1,  0,  2,  0,  6,  5,  1,  5,  9,  1, 10,  5,  9,  1,  0,  5,
        1, 10,  1,  5,  5,  1, 11, 10,  5,  5,  5,  5,  5,  5,  5,  1,  2,
        9, 11, 10,  1, 10, 12,  5,  1,  5, 14, 12,  1,  2,  2, 11, 11,  0,
        1,  6,  2, 12,  5,  9,  2, 11, 15,  2,  5,  5,  5])

In [18]:
np.array(pred_data.data['test']['report'][:64])

array([ 3,  2,  1,  7,  1,  7,  3,  2, 12, 10,  3, 14,  6, 10,  3,  1,  6,
        2, 14,  2, 15,  6,  3, 12, 10,  6,  3,  6,  6,  6,  6,  6,  2,  7,
       10, 12, 11,  2, 11, 13,  6,  2,  3, 15, 13,  2,  9,  3, 12, 12,  1,
        2,  7, 10, 13,  6, 10,  3, 12, 16,  9,  6,  6,  6])

In [19]:
class GetResult(BaseTransformer):
    def __init__(self, config=None):
        if config is None:
            self.config = {'op_type': 'transformer',
                           'name': 'Resulter',
                           'request_names': ['predicted_test'],
                           'new_names': ['test']}
        else:
            self.config = config

        super().__init__(self.config)

    def _transform(self, dataset):
        request, report = dataset.main_names

        pred_name = self.config['request_names'][0]
        print(pred_name)
        real_name = self.config['new_names'][0]
        print(real_name)
        pred_data = dataset.data[pred_name]
        real_data = np.array(dataset.data[real_name][report])

        preds = pred_data[0]
        for x in pred_data[1:]:
            preds = np.concatenate((preds, x), axis=0)

        preds = np.argmax(preds, axis=1)
        for i, x in enumerate(preds):
            preds[i] = x + 1

        preds = preds[:len(real_data)]

        results = get_result(preds, real_data)
        dataset.data['results'] = results

        conf = dataset.pipeline_config
        date = dataset.date
        logging(results, conf, date)

        return dataset

end_pipe = Pipeline([(GetResult,)])
end_res = end_pipe.run(pred_data)

predicted_test
test


/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
end_res.data['results']

{'accuracy': 'accuracy : 0.7640223463687151',
 'classes': {'1': {'f1': 0.0,
   'number_test_objects': 0,
   'precision': 0.0,
   'recall': 0.0},
  '10': {'f1': 0.35398230088495575,
   'number_test_objects': 90,
   'precision': 0.8695652173913043,
   'recall': 0.2222222222222222},
  '11': {'f1': 0.8212121212121212,
   'number_test_objects': 349,
   'precision': 0.8713826366559485,
   'recall': 0.7765042979942693},
  '12': {'f1': 0.6717171717171717,
   'number_test_objects': 203,
   'precision': 0.689119170984456,
   'recall': 0.6551724137931034},
  '13': {'f1': 0.7402597402597402,
   'number_test_objects': 220,
   'precision': 0.7066115702479339,
   'recall': 0.7772727272727272},
  '14': {'f1': 0.7837445573294629,
   'number_test_objects': 345,
   'precision': 0.7848837209302325,
   'recall': 0.782608695652174},
  '15': {'f1': 0.0,
   'number_test_objects': 20,
   'precision': 0.0,
   'recall': 0.0},
  '16': {'f1': 0.8380952380952381,
   'number_test_objects': 118,
   'precision': 0.956

In [21]:
# config = get_cnn_conf(path_1, emb_path)
# dataset = init_dataset()
# WCNN = GetCNN(CNN, config)
# neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (WCNN,), (GetResult,)]
# pipeline_1 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_1.run(dataset)

In [22]:
# config = get_cnn_conf(path_1, emb_path)
# dataset = init_dataset()
# dCNN = GetCNN(DCNN, config)
# neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (dCNN,), (GetResult,)]
# pipeline_2 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_2.run(dataset)

# Pipelines with Linear Models

In [23]:
conf_0 = {'op_type': 'model', 'name': 'Linear Regression',
          'fit_names': ['train_vec'], 'new_names': ['predicted_test'],
          'predict_names': ['test_vec']}

## Run pipeline with linear regression

In [24]:
# (Tokenizer, tok_conf), (Lemmatizer,), (concat, None),
# dataset = init_dataset()
# LR = skmodel(LogisticRegression, conf_0)

# pipe_0 = [(tfidf_, tfidf_conf_2), (LR,), (GetResultLinear_W,)]
# pipeline_0 = Pipeline(pipe_0, mode='train', output=None)
# pipeline_1 = Pipeline(pipe_0, mode='infer', output='dataset')
# res = pipeline_1.run(dataset)

## Run pipeline with SVC

In [25]:
# dataset = init_dataset()

# conf_0['name'] = 'LinearSVC'
# conf_0['op_type'] = 'model'
# LSVC = skmodel(LinearSVC, conf_0)

# # (Speller, spl_conf),

# pipe_1 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, tfidf_conf_2),
#           (LSVC,), (GetResultLinear_W,)]
# pipeline_2 = Pipeline(pipe_1, mode='train', output=None)
# pipeline_3 = Pipeline(pipe_1, mode='infer', output='dataset')

# res = pipeline_3.run(dataset)

##  Run pipeline with Random Forest

In [26]:
# dataset = init_dataset()

# conf_0['name'] = 'RandomForestClassifier'
# conf_0['op_type'] = 'model'
# RFC = skmodel(RandomForestClassifier, conf_0)


# # (Speller, spl_conf), 
# pipe_2 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, tfidf_conf_2),
#           (RFC,), (GetResultLinear_W,)]
# pipeline_4 = Pipeline(pipe_2, mode='train', output=None)
# pipeline_5 = Pipeline(pipe_2, mode='infer', output='dataset')

# res = pipeline_5.run(dataset)

## Run pipeline with GBM

In [27]:
# from os.path import join

# date = datetime.datetime.now()
# dataset_name = 'vkusvill'
# language = 'russian'
# file_path = join('./data', language, dataset_name, 'data', 'vkusvill_all_categories.csv')

# dataset = init_dataset_tiny(file_path, language, dataset_name, date)

# conf_0['name'] = 'LGBMClassifier'
# conf_0['op_type'] = 'model'
# LGBM = skmodel(LGBMClassifier, conf_0)

# # (Speller, spl_conf),
# pipe_3 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, ),
#           (LGBM,), (GetResultLinear_W,)]
# pipeline_6 = Pipeline(pipe_3, mode='train', output=None)
# pipeline_7 = Pipeline(pipe_3, mode='infer', output='dataset')

# res = pipeline_7.run(dataset)